In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
model_id = "meta-llama/Llama-2-7b-chat-hf"

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.use_default_system_prompt = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
CHROMA_PATH = "chroma_db"

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

In [5]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

In [6]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/opt/anaconda3/envs/qaenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [7]:
from langchain_chroma import Chroma

In [8]:
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

In [9]:
query_text = 'Why is numpy fast?'

In [10]:
results = db.similarity_search_with_relevance_scores(query_text, k=3)

In [11]:
results

[(Document(metadata={'page': 7, 'source': 'data/numpy-user.pdf', 'start_index': 798}, page_content='Python. Indeed, the NumPy idiom is even simpler! This last example illustrates two of NumPy’s features which are\nthe basis of much of its power: vectorization and broadcasting.\n1.1.1 Why is NumPy Fast?\nVectorization describes the absence of any explicit looping, indexing, etc., in the code - these things are taking place,\nof course, just “behind the scenes” in optimized, pre-compiled C code. Vectorized code has many advantages, among\nwhich are:'),
  0.3488694598451352),
 (Document(metadata={'page': 160, 'source': 'data/numpy-user.pdf', 'start_index': 0}, page_content='NumPy User Guide, Release 1.18.4\nAfter processing data at the current element of the array, the next element of the array can be obtained using the macro\nPyArray_ITER_NEXT (iter ). The iteration always proceeds in a C-style contiguous fashion (last index varying\nthe fastest). The PyArray_ITER_GOTO (iter ,destination

In [12]:
context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])

In [13]:
from langchain.prompts import ChatPromptTemplate

In [14]:
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query_text)

In [16]:
input_ids = tokenizer.encode(prompt, return_tensors="pt")

output = model.generate(input_ids, max_length=512, num_beams=4, no_repeat_ngram_size=2)

response = tokenizer.decode(output[0], skip_special_tokens=True)

Both `max_new_tokens` (=200) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


KeyboardInterrupt: 